## Getting data from bucket as csv:

In [244]:
from dotenv import load_dotenv
import boto3
import os

load_dotenv()
access_key_id = os.environ.get("OLIVER_ACCESS_KEY")
access_key_secret = os.environ.get("OLIVER_SECRET_ACCESS_KEY")

session = boto3.Session(
    aws_access_key_id=access_key_id,
    aws_secret_access_key=access_key_secret,
)

s3_client = boto3.client(
    's3',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=access_key_secret
)

bucket = session.resource('s3').Bucket(name='emu-air-quality-bucket')

latest_object = list(bucket.objects.all())[-1]

with open("./download.csv", 'wb') as file:
    s3_client.download_fileobj(
        bucket.name,
        latest_object.key,
        file
    )





## Grabbing data from csv as DataFrame:

In [245]:
import pandas as pd
import re
import numpy as np

raw_data = pd.read_csv("./download.csv", dtype=str)

def get_application_id(val: str) -> str:
    my_regex = re.compile(r'.*\{application_id=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return result

def get_eui(val: str) -> str:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*eui=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return result

def get_co2(val: str) -> int:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*co2=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return int(result)

def get_hcho(val: str) -> float:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*hcho=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return float(result)

def get_humidity(val: str) -> float:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*humidity=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return float(result)

def get_light_level(val: str) -> int:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*light_level=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return int(result)

def get_pir(val: str) -> str:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*pir=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return result

def get_pm10(val: str) -> int:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*pm10=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return int(result)

def get_pm2_5(val: str) -> int:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*pm2_5=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return int(result)

def get_pressure(val: str) -> float:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*pressure=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return float(result)

def get_temperature(val: str) -> float:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*temperature=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return float(result)

def get_tvoc(val: str) -> int:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*tvoc=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return int(result)

def get_measured_at(val: str) -> str:
    if "decoded_payload=null" in val:
        return np.nan
    my_regex = re.compile(r'.*received_at=(.*?)[\}\,]')
    result = my_regex.search(val).group(1)
    if result == "null":
        return np.nan
    return result



conversion_dict = {
    "received_at": {
        "column": "received_at",
        "function": lambda val: val
    },
    "application_id": {
        "column": "end_device_ids",
        "function": get_application_id
    },
    "eui": {
        "column": "uplink_message",
        "function": get_eui
    },
    "co2": {
        "column": "uplink_message",
        "function": get_co2
    },
    "hcho": {
        "column": "uplink_message",
        "function": get_hcho
    },
    "humidity": {
        "column": "uplink_message",
        "function": get_humidity
    },
    "light_level": {
        "column": "uplink_message",
        "function": get_light_level
    },
    "pir": {
        "column": "uplink_message",
        "function": get_pir
    },
    "pm10": {
        "column": "uplink_message",
        "function": get_pm10
    },
    "pm2_5": {
        "column": "uplink_message",
        "function": get_pm2_5
    },
    "pressure": {
        "column": "uplink_message",
        "function": get_pressure
    },
    "temperature": {
        "column": "uplink_message",
        "function": get_temperature
    },
    "tvoc": {
        "column": "uplink_message",
        "function": get_tvoc
    },
    "measured_at": {
        "column": "uplink_message",
        "function": get_measured_at
    },
}

data = pd.DataFrame({ key: raw_data[val["column"]].apply(val["function"]) for key, val in conversion_dict.items() })


In [246]:
data.dropna().sort_values(by="measured_at").tail(50)

,received_at,application_id,eui,co2,hcho,humidity,light_level,pir,pm10,pm2_5,pressure,temperature,tvoc,measured_at
377,2023-02-28T02:56:21.990167528Z,office-air-quality,AC1F09FFFE053AD4,416.0,0.01,55.0,1.0,trigger,6.0,6.0,1011.0,29.5,25.0,2023-02-28T02:56:21.748981608Z
225,2023-02-28T03:01:21.922949217Z,office-air-quality,AC1F09FFFE053AD4,401.0,0.01,55.0,1.0,idle,9.0,8.0,1011.0,29.4,42.0,2023-02-28T03:01:21.712642836Z
23,2023-02-28T03:06:22.136282886Z,office-air-quality,AC1F09FFFE053AD4,403.0,0.01,54.5,1.0,idle,5.0,5.0,1010.9,29.4,25.0,2023-02-28T03:06:21.931255755Z
24,2023-02-28T03:11:22.033350465Z,office-air-quality,AC1F09FFFE053AD4,405.0,0.01,54.5,1.0,idle,6.0,6.0,1011.0,29.3,39.0,2023-02-28T03:11:21.716571127Z
268,2023-02-28T03:16:21.996626912Z,office-air-quality,AC1F09FFFE053AD4,408.0,0.01,54.5,1.0,idle,4.0,4.0,1010.9,29.4,53.0,2023-02-28T03:16:21.767278676Z
242,2023-02-28T03:21:22.017987331Z,office-air-quality,AC1F09FFFE053AD4,396.0,0.01,54.5,1.0,idle,5.0,5.0,1010.9,29.3,35.0,2023-02-28T03:21:21.807955755Z
224,2023-02-28T03:26:21.964095513Z,office-air-quality,AC1F09FFFE053AD4,406.0,0.01,54.5,1.0,idle,5.0,5.0,1010.9,29.3,34.0,2023-02-28T03:26:21.754844754Z
27,2023-02-28T03:31:22.016566001Z,office-air-quality,AC1F09FFFE053AD4,409.0,0.01,54.5,1.0,idle,6.0,6.0,1010.9,29.4,49.0,2023-02-28T03:31:21.806670256Z
338,2023-02-28T03:36:21.941871603Z,office-air-quality,AC1F09FFFE053AD4,408.0,0.01,54.5,1.0,idle,6.0,6.0,1010.8,29.5,82.0,2023-02-28T03:36:21.736819085Z
187,2023-02-28T03:46:22.027175967Z,office-air-quality,AC1F09FFFE053AD4,397.0,0.01,54.0,1.0,trigger,5.0,5.0,1010.7,29.5,23.0,2023-02-28T03:46:21.818422919Z


## Getting data fro a datastore/modifying a pipe

We don't need to use the S3 Bucket, since I can just set a lambda on a chron job to access the datastore and push to the database.

In [249]:

analytics_client = boto3.client(
    'iotanalytics',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=access_key_secret
)

response = analytics_client.describe_pipeline(
    pipelineName='emu_air_quality_pipe'
)
print(response)

response = analytics_client.get_dataset_content(
    datasetName="emu_air_quality_data",
    versionId="$LATEST_SUCCEEDED"
)
print(response)

{'ResponseMetadata': {'RequestId': '97b0248e-8121-4760-b92b-a243acc3edb6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 28 Feb 2023 07:27:48 GMT', 'content-type': 'application/json', 'content-length': '747', 'connection': 'keep-alive', 'x-amzn-requestid': '97b0248e-8121-4760-b92b-a243acc3edb6', 'cache-control': 'no-cache, no-store, must-revalidate, private', 'expires': '0', 'pragma': 'no-cache'}, 'RetryAttempts': 0}, 'pipeline': {'name': 'emu_air_quality_pipe', 'arn': 'arn:aws:iotanalytics:us-east-1:638201302569:pipeline/emu_air_quality_pipe', 'activities': [{'channel': {'name': 'MyInput', 'channelName': 'emu_air_quality', 'next': 'MyOutput'}}, {'datastore': {'name': 'MyOutput', 'datastoreName': 'emu_air_quality_store'}}], 'reprocessingSummaries': [], 'creationTime': datetime.datetime(2023, 2, 27, 14, 7, 29, 788000, tzinfo=tzlocal()), 'lastUpdateTime': datetime.datetime(2023, 2, 28, 16, 49, 25, 763000, tzinfo=tzlocal())}}
{'ResponseMetadata': {'RequestId': '2f4e138f-e0f1-4d9a-b

## Creating a dynamodb table

In [248]:
db_client = boto3.client(
    'dynamodb',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=access_key_secret
)

response = db_client.create_table(
    AttributeDefinitions=[
        {
            'AttributeName': 'DeviceEUI',
            'AttributeType': 'S',
        },
        {
            'AttributeName': 'Timestamp',
            'AttributeType': 'S',
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'DeviceEUI',
            'KeyType': 'HASH',
        },
        {
            'AttributeName': 'Timestamp',
            'KeyType': 'RANGE',
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5,
    },
    TableName='DeviceMeasurements',
)

print(response)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'DeviceEUI', 'AttributeType': 'S'}, {'AttributeName': 'Timestamp', 'AttributeType': 'S'}], 'TableName': 'DeviceMeasurements', 'KeySchema': [{'AttributeName': 'DeviceEUI', 'KeyType': 'HASH'}, {'AttributeName': 'Timestamp', 'KeyType': 'RANGE'}], 'TableStatus': 'CREATING', 'CreationDateTime': datetime.datetime(2023, 2, 28, 17, 26, 9, 903000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:us-east-1:638201302569:table/DeviceMeasurements', 'TableId': 'ccbc5f52-4087-417b-9418-8edecb4ecea7'}, 'ResponseMetadata': {'RequestId': '38ROP6LQN5D76O2KHP4RHC852JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Tue, 28 Feb 2023 07:26:10 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '608', 'connection': 'keep-alive', 'x-amzn-requ

## Up next:
- write a lambda to hit the datastore, parse the data, and push to the dynamodb table
- give luisa's key permission to act on dynamodb tables
- update luisa with new and better code to access the db